# Challenge Data Engineer LATAM - Juan Felipe Hurtado

- El archivo json se tiene en el directorio raiz de este archivo.

# Importaciones globales del reto:

In [101]:
import os
import pandas as pd
import ujson as json
import emoji
import re
import cProfile
import pstats
from memory_profiler import profile
from collections import Counter
from typing import List, Tuple
from datetime import datetime
from collections import defaultdict

# 1 Las top 10 fechas donde hay más tweets. 
Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.
- Enfoque: Tiempo de ejecucion optimizado.

- Descripción:
Este código define una función llamada q1_time que procesa un archivo. Si existe un archivo Parquet más reciente que el archivo original, lo lee; de lo contrario, lee el archivo JSON, realiza manipulaciones de datos y lo guarda como Parquet. Luego, identifica las 10 fechas con más apariciones en el archivo original y, para cada una de ellas, encuentra el nombre de usuario con más apariciones en esa fecha. El resultado final es una lista de tuplas que contienen la fecha y el nombre de usuario correspondiente para las 10 fechas principales.

In [102]:
def q1_time(archivo_json: str) -> List[Tuple[datetime.date, str]]:
    # Separar el nombre del archivo y su extensión
    nombre_archivo, _ = os.path.splitext(archivo_json)
    archivo_parquet = f"{nombre_archivo}.parquet"
    
    # Verificar si existe un archivo parquet más reciente que el JSON
    if os.path.exists(archivo_parquet) and os.path.getmtime(archivo_parquet) >= os.path.getmtime(archivo_json):
        # Si existe y es más reciente, cargar datos desde el archivo parquet
        df = pd.read_parquet(archivo_parquet)
    else:
        # Si no existe un archivo parquet o es más antiguo, cargar datos desde el JSON
        df = pd.read_json(archivo_json, lines=True)
        # Optimizar la conversión de fecha y extracción de nombre de usuario
        df['date'] = pd.to_datetime(df['date']).dt.date
        df['username'] = df['user'].apply(lambda x: x.get('username'))
        df.drop(columns=['user'], inplace=True)
        # Guardar los datos procesados en formato parquet
        df.to_parquet(archivo_parquet, index=False)

    # Obtener el usuario con más tweets para cada fecha de manera eficiente
    top_users_df = df.groupby(['date', 'username']).size().reset_index(name='count')
    top_users_df = top_users_df.loc[top_users_df.groupby('date')['count'].idxmax()]
    
    # Encontrar las 10 fechas con más tweets y los usuarios correspondientes de manera eficiente
    top_dates_df = df['date'].value_counts().nlargest(10).reset_index()
    result = [(fecha, top_users_df[top_users_df['date'] == fecha]['username'].values[0]) for fecha in top_dates_df['index']]
    
    return result

 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q1_memory que procesa un archivo de texto. Lee cada línea del archivo, interpreta cada línea como un objeto JSON que representa un tweet, y realiza un seguimiento de la fecha y el nombre de usuario más frecuentes para cada fecha. Luego, encuentra las 10 fechas con mayor número de usuarios únicos y devuelve una lista de tuplas que contienen la fecha y el nombre de usuario más frecuente para cada una de esas fechas.

In [103]:


def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Crear un diccionario anidado para contar usuarios por fecha
    usuarios_por_fecha = defaultdict(lambda: defaultdict(int))
    # Abrir el archivo en modo lectura
    with open(file_path, "r") as archivo:
        # Iterar sobre cada línea en el archivo
        for linea in archivo:
            # Cargar cada línea como un objeto JSON
            tweet = json.loads(linea)
            # Extraer la fecha del tweet y convertirla en un objeto de fecha
            fecha_tweet_str = tweet['date'].split("T")[0]
            fecha_tweet = datetime.strptime(fecha_tweet_str, '%Y-%m-%d').date()
            # Obtener el nombre de usuario del tweet
            nombre_usuario = tweet['user']['username']
            # Actualizar el contador de usuarios por fecha
            usuarios_por_fecha[fecha_tweet][nombre_usuario] += 1
    # Inicializar una lista vacía para almacenar las fechas y usuarios principales
    fechas_usuarios_principales = []
    # Iterar a través de las fechas y sus recuentos de usuarios
    for fecha, recuentos_usuarios in usuarios_por_fecha.items():
        # Encontrar el usuario con el recuento máximo para cada fecha
        usuario_principal = max(recuentos_usuarios, key=recuentos_usuarios.get)
        # Agregar la fecha y el usuario principal a la lista
        fechas_usuarios_principales.append((fecha, usuario_principal))
    # Ordenar la lista por recuento de usuarios en orden descendente y tomar las 10 primeras entradas
    resultado = sorted(fechas_usuarios_principales, key=lambda x: usuarios_por_fecha[x[0]][x[1]], reverse=True)[:10]

    return resultado


# 2. Los top 10 emojis más usados con su respectivo conteo.
- Enfoque: Tiempo de ejecucion optimizado
- Descripción: Este código define una función llamada q2_time que procesa un archivo. Primero, verifica si existe un archivo Parquet con el mismo nombre que el archivo JSON de entrada. Si existe, lo lee; de lo contrario, lee el archivo JSON, extrae los emojis de los contenidos de los tweets, cuenta la frecuencia de cada emoji y devuelve una lista de los 10 emojis más comunes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el emoji y su frecuencia de aparición en los tweets del archivo.

In [104]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    # Crea el nombre del archivo Parquet reemplazando la extensión .json por .parquet
    archivo_parquet = file_path.replace('.json', '.parquet')
    
    # Verifica si existe el archivo Parquet
    if os.path.exists(archivo_parquet):
        # Lee el archivo Parquet si existe
        df = pd.read_parquet(archivo_parquet)
    else:
        # Lee el archivo JSON si el Parquet no existe y lo guarda como Parquet
        df = pd.read_json(file_path, lines=True)
        df.to_parquet(archivo_parquet, index=False)
    
    # Inicializa una lista vacía para almacenar todos los emojis en el contenido de los tweets
    todos_emojis = []
    
    # Itera a través de la columna 'content' del DataFrame
    for contenido in df['content']:
        # Extrae los emojis de cada contenido y los agrega a la lista de todos los emojis
        emojis_en_contenido = [entrada['emoji'] for entrada in emoji.emoji_list(contenido)]
        todos_emojis.extend(emojis_en_contenido)
    
    # Cuenta la frecuencia de cada emoji
    conteo_emojis = Counter(todos_emojis)
    
    # Encuentra los 10 emojis más comunes
    top_emojis = conteo_emojis.most_common(10)
    
    return top_emojis


 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q2_memory que procesa un archivo de texto. Lee cada línea del archivo, interpreta cada línea como un objeto JSON que representa un tweet y cuenta la frecuencia de cada emoji en el contenido de los tweets. Luego, devuelve una lista de los 10 emojis más comunes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el emoji y su frecuencia de aparición en los tweets del archivo.

In [105]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    # Inicializa un contador para contar emojis
    conteo_emojis = Counter()
    
    # Abre el archivo en modo lectura
    with open(file_path, 'r') as archivo:
        # Itera sobre cada línea en el archivo
        for linea in archivo:
            # Carga cada línea como un objeto JSON
            tweet = json.loads(linea)
            
            # Obtiene el contenido del tweet o una cadena vacía si no hay contenido
            contenido = tweet.get('content', '')
            
            # Extrae los emojis del contenido del tweet y actualiza el contador de emojis
            emojis_en_contenido = [entrada['emoji'] for entrada in emoji.emoji_list(contenido)]
            conteo_emojis.update(emojis_en_contenido)
    
    # Encuentra los 10 emojis más comunes
    top_emojis = conteo_emojis.most_common(10)
    
    return top_emojis


# 3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. 
- Enfoque: Tiempo de ejecucion optimizado
- Descripción: Este código define una función llamada q3_time que procesa un archivo. Primero, verifica si existe un archivo Parquet con el mismo nombre que el archivo JSON de entrada. Si existe, lo lee; de lo contrario, lee el archivo JSON, encuentra menciones en el contenido de los tweets, cuenta la frecuencia de cada mención y devuelve una lista de las 10 menciones más frecuentes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el nombre de usuario mencionado y la frecuencia de aparición en los tweets del archivo.

In [106]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # Crea el nombre del archivo Parquet reemplazando la extensión .json por .parquet
    archivo_parquet = file_path.replace('.json', '.parquet')
    
    # Verifica si existe el archivo Parquet
    if os.path.exists(archivo_parquet):
        # Lee el archivo Parquet si existe
        df = pd.read_parquet(archivo_parquet)
    else:
        # Lee el archivo JSON si el Parquet no existe y lo guarda como Parquet
        df = pd.read_json(file_path, lines=True)
        df.to_parquet(archivo_parquet, index=False)
    
    # Encuentra menciones en el contenido de los tweets y crea una nueva columna 'mentions'
    df['menciones'] = df['content'].str.findall(r'@(\w+)')
    
    # Divide las filas del DataFrame por cada mención, creando múltiples filas duplicadas
    df = df.explode('menciones', ignore_index=True)
    
    # Calcula la frecuencia de menciones y crea un nuevo DataFrame con columnas 'nombre_de_usuario' y 'conteo'
    conteo_menciones = df['menciones'].value_counts().reset_index()
    conteo_menciones.columns = ['nombre_de_usuario', 'conteo']
    
    # Obtiene las 10 menciones más frecuentes como un iterable de tuplas sin índice
    top_menciones = conteo_menciones.head(10).itertuples(index=False, name=None)
    
    # Convierte el iterable en una lista de tuplas
    return list(top_menciones)


 - Enfoque: Memoria en uso optimizada
 - Descripción: Este código define una función llamada q3_memory que procesa un archivo de texto. Lee cada línea del archivo, intenta cargarla como un objeto JSON (capturando excepciones JSONDecodeError para líneas inválidas), busca menciones en el contenido de los tweets y cuenta la frecuencia de cada mención. Luego, devuelve una lista de las 10 menciones más frecuentes junto con sus recuentos. El resultado final es una lista de tuplas que contienen el nombre de usuario mencionado y la frecuencia de aparición en los tweets del archivo.

In [107]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    # Inicializa un contador para contar menciones
    conteo_menciones = Counter()
    
    # Abre el archivo en modo lectura
    with open(file_path, 'r') as archivo:
        # Itera sobre cada línea en el archivo
        for linea in archivo:
            try:
                # Intenta cargar cada línea como un objeto JSON (puede haber excepciones JSONDecodeError)
                tweet = json.loads(linea)
                
                # Obtiene el contenido del tweet o una cadena vacía si no hay contenido
                contenido = tweet.get('content', '')
                
                # Encuentra menciones en el contenido del tweet
                menciones = re.findall(r'@(\w+)', contenido)
                
                # Actualiza el contador de menciones con las menciones encontradas
                conteo_menciones.update(menciones)
            except json.JSONDecodeError:
                # Captura excepciones JSONDecodeError en caso de líneas inválidas
                pass
    
    # Obtiene las 10 menciones más frecuentes
    top_menciones = conteo_menciones.most_common(10)
    
    return top_menciones


# Resultados


# Question 1

In [108]:
q1_time("./farmers-protest-tweets-2021-2-4.json")

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [109]:
q1_memory("./farmers-protest-tweets-2021-2-4.json")

[(datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist')]

# Question 2

In [110]:
q2_time("./farmers-protest-tweets-2021-2-4.json")

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

In [111]:
q2_memory("./farmers-protest-tweets-2021-2-4.json")

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

# Question 3

In [112]:
q3_time("./farmers-protest-tweets-2021-2-4.json")

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

In [113]:
q3_memory("./farmers-protest-tweets-2021-2-4.json")

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

# Medición y evaluación de las funciones

# Question 1:

In [114]:
# Carga la extensión memory_profiler para evaluar el uso de memoria en las funciones
%load_ext memory_profiler

# Evaluar el uso de memoria de la función q1_time
print("Evaluación de memoria q1_time")
%memit q1_time("./farmers-protest-tweets-2021-2-4.json")

# Evaluar el uso de memoria de la función q1_memory
print("Evaluación de memoria q1_memory")
%memit q1_memory("./farmers-protest-tweets-2021-2-4.json")

# Inicializar el profiler de cProfile para evaluar el rendimiento del código
profiler = cProfile.Profile()
profiler.enable()

# Llamar a las funciones q1_memory y q1_time para medir el rendimiento
q1_memory("./farmers-protest-tweets-2021-2-4.json")
q1_time("./farmers-protest-tweets-2021-2-4.json")

# Deshabilitar el profiler cProfile y guardar estadísticas en un archivo
profiler.disable()
profiler.dump_stats("output.pstats1")

# Crear una instancia de Stats de pstats para analizar las estadísticas
stats = pstats.Stats("output.pstats1")

# Ordenar y mostrar las estadísticas según el tiempo acumulativo (cumulative) de las 10 funciones principales
stats.sort_stats("cumulative")
stats.print_stats(10)


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Evaluación de memoria q1_time
peak memory: 3458.63 MiB, increment: 189.10 MiB
Evaluación de memoria q1_memory
peak memory: 3388.33 MiB, increment: 0.41 MiB
Tue Oct  3 04:12:28 2023    output.pstats1

         3286184 function calls (3285917 primitive calls) in 8.495 seconds

   Ordered by: cumulative time
   List reduced from 879 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    8.495    2.832 C:\Users\Juan Hurtado\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3469(run_code)
        3    0.000    0.000    8.495    2.832 {built-in method builtins.exec}
        1    1.328    1.328    6.709    6.709 C:\Users\Juan Hurtado\AppData\Local\Temp\ipykernel_35976\2436507414.py:1(q1_memory)
   117407    2.362    0.000    2.362    0.000 {built-in method ujson.loads}
   117407    0.074  

# Question 2:

In [115]:
# Carga la extensión memory_profiler para evaluar el uso de memoria en las funciones
%load_ext memory_profiler

# Evaluar el uso de memoria de la función q2_time
print("Evaluación de memoria q2_time")
%memit q2_time("./farmers-protest-tweets-2021-2-4.json")

# Evaluar el uso de memoria de la función q2_memory
print("Evaluación de memoria q2_memory")
%memit q2_memory("./farmers-protest-tweets-2021-2-4.json")

# Inicializar el profiler de cProfile para evaluar el rendimiento del código
profiler = cProfile.Profile()
profiler.enable()

# Llamar a las funciones q2_memory y q2_time para medir el rendimiento
q2_memory("./farmers-protest-tweets-2021-2-4.json")
q2_time("./farmers-protest-tweets-2021-2-4.json")

# Deshabilitar el profiler cProfile y guardar estadísticas en un archivo
profiler.disable()
profiler.dump_stats("output.pstats2")

# Crear una instancia de Stats de pstats para analizar las estadísticas
stats = pstats.Stats("output.pstats2")

# Ordenar y mostrar las estadísticas según el tiempo acumulativo (cumulative) de las 10 funciones principales
stats.sort_stats("cumulative")
stats.print_stats(10)


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Evaluación de memoria q2_time
peak memory: 3505.53 MiB, increment: 89.42 MiB
Evaluación de memoria q2_memory
peak memory: 3434.79 MiB, increment: 0.06 MiB
Tue Oct  3 04:14:17 2023    output.pstats2

         138721680 function calls (138721667 primitive calls) in 70.338 seconds

   Ordered by: cumulative time
   List reduced from 400 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000   79.898   26.633 C:\Users\Juan Hurtado\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3469(run_code)
        3    0.000    0.000   79.898   26.633 {built-in method builtins.exec}
   234814    0.176    0.000   72.786    0.000 c:\Users\Juan Hurtado\AppData\Local\Programs\Python\Python311\Lib\site-packages\emoji\core.py:282(emoji_list)
   234814   12.642    0.000   72.610    0.000 c:\Users\Juan Hurtad

# Question 3:

In [116]:
# Carga la extensión memory_profiler para evaluar el uso de memoria en las funciones
%load_ext memory_profiler

# Evaluar el uso de memoria de la función q3_time
print("Evaluación de memoria q3_time")
%memit q3_time("./farmers-protest-tweets-2021-2-4.json")

# Evaluar el uso de memoria de la función q3_memory
print("Evaluación de memoria q3_memory")
%memit q3_memory("./farmers-protest-tweets-2021-2-4.json")

# Inicializar el profiler de cProfile para evaluar el rendimiento del código
profiler = cProfile.Profile()
profiler.enable()

# Llamar a las funciones q3_memory y q3_time para medir el rendimiento
q3_memory("./farmers-protest-tweets-2021-2-4.json")
q3_time("./farmers-protest-tweets-2021-2-4.json")

# Deshabilitar el profiler cProfile y guardar estadísticas en un archivo
profiler.disable()
profiler.dump_stats("output.pstats3")

# Crear una instancia de Stats de pstats para analizar las estadísticas
stats = pstats.Stats("output.pstats3")

# Ordenar y mostrar las estadísticas según el tiempo acumulativo (cumulative) de las 10 funciones principales
stats.sort_stats("cumulative")
stats.print_stats(10)


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
Evaluación de memoria q3_time
peak memory: 3612.97 MiB, increment: 158.05 MiB
Evaluación de memoria q3_memory
peak memory: 3470.02 MiB, increment: 0.03 MiB
Tue Oct  3 04:14:31 2023    output.pstats3

         1399510 function calls (1399366 primitive calls) in 6.752 seconds

   Ordered by: cumulative time
   List reduced from 838 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    6.767    2.256 C:\Users\Juan Hurtado\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3469(run_code)
        3    0.000    0.000    6.767    2.256 {built-in method builtins.exec}
        1    1.122    1.122    4.697    4.697 C:\Users\Juan Hurtado\AppData\Local\Temp\ipykernel_35976\1862612398.py:1(q3_memory)
   117407    2.235    0.000    2.235    0.000 {built-in method ujson.loads}
        1    0.013  